In [ ]:
import os
GPU_id = 4
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_id)
import cudf
import numpy as np
import nvtabular as nvt
from cudf.io.parquet import ParquetWriter

data_path = '/datasets/criteo/crit_orig/'
df_train = ''
df_valid = 'valid/'
df_test = 'test/'

train_set = [data_path + df_train + x for x in os.listdir(data_path + df_train) if x.startswith("day_") and not x.endswith("parquet")] 
train_set.sort()

cont_names = ["I" + str(x) for x in range(1, 14)]
cat_names = ["C" + str(x) for x in range(1, 27)]
cols = ["label"] + cont_names + cat_names


dtypes = {}
dtypes["label"] = np.int64
for x in cont_names:
    dtypes[x] = np.int64
for x in cat_names:
    dtypes[x] = np.object
    
    
def file_to_pq(target_files, file_type, output_folder='par_out', cols=[], dtypes=None):
    target_files.sort()
    tar = nvt.dataset(target_files, engine=file_type, names=cols, gpu_memory_frac=0.05, sep='\t', dtypes=dtypes)
    count = 0
    old_file_path = None
    writer = None
    for gdf in tar:
        #gdf.to_parquet(output_folder)
        file_path = os.path.join(output_folder, os.path.split(tar.itr.file_path)[1].split('.')[0])
        path = f'{file_path}.parquet'
        if file_path != old_file_path:
            writer = ParquetWriter(path)
        writer.write_table(gdf)
        old_file_path = file_path
        del gdf
path_out = '/raid/criteo/tests/jp_csv_orig/'
file_to_pq(train_set, 'csv', output_folder=path_out, cols=cols, dtypes=dtypes)